## Homework 3 - Interactive viz

In [1]:
from handle_data import *
import pandas as pd
import numpy as np
import folium

### 1) Choropleth map

The goal of the exercise is to build a Choropleth map which shows how much grant money goes to each Swiss canton. In order to do that we work with  P3 database of the SNSF (Swiss National Science Foundation).


To do so we proceed in this direction:

__a__. Create the Choropleth map
 * Go through the `.json` to get some information about cantons
 * Create a "*fake*" `DataFrame` and use it to overlay the map with the colored one
 
 
__b__. Fill in the map with the correct values
 * Extract and filter data
 * Use `GoogleMaps` API to get the canton of each *University*
 * Fill the correct data into the already created `DataFrame`
 * Obtain the Choropleth map

#### a) Create the Choropleth map

The first thing we do is to parse the provided `ch-cantons.topojson.json` (stored in `TopoJSON` dir). It would be useful to get the `Ids` of the cantons. The used function is stored in the [library](parse_topojson.py)

In [2]:
# Declare the file's path
topo_path = 'TopoJSON/ch-cantons.topojson.json'

# Get sorted list of cantons' ids
list_cantons = parse_topojson(topo_path)

Once we get the list of the all cantons, we create the *fake* `DataFrame` that will be used to create our beloved map. 
According to what the `.choropleth()` method of `Folium` requires, we setup the `df` as follows:

| Canton | Total grants |
|:------:|:------------:|
|   AG   |      19      |
|   AI   |      39      |
|   ...  |      ...     |
|   ...  |      ...     |
|   ZH   |      100     |

*Remark*: for this first step, the data contained in the `Total grants` column are not the true one.


In [6]:
# Create the fake 'Total grants' column 
total_grants = list(range(26))

# Create dataframe
df_cantons = create_map_df(list_cantons, total_grants)

,Cantons,Total grants
0,AG,0
1,AI,1
2,AR,2
3,BE,3
4,BL,4
5,BS,5
6,FR,6
7,GE,7
8,GL,8
9,GR,9


In [ ]:
#cantons_data = pd.DataFrame(df_data)

In [ ]:
switzerland_map = folium.Map(location=[46.912457, 8.191891],
                   tiles='Mapbox Bright', zoom_start=2)

In [ ]:
switzerland_map.choropleth(geo_path=topo_path, topojson='objects.antarctic_ice_shelf')

In [ ]:
(self, geo_path=None, geo_str=None, data_out='data.json',
                   data=None, columns=None, key_on=None, threshold_scale=None,
                   fill_color='blue', fill_opacity=0.6, line_color='black',
                   line_weight=1, line_opacity=1, legend_name="",
                   topojson=None, reset=False, smooth_factor=None,
                   highlight=None)

In [ ]:
ice_map.save('ice_map.html')

In [ ]:
data = pd.read_csv('Data/P3_GrantExport.csv', sep = ';')

In [ ]:
data.head()

In [ ]:
# Get the df that contains those projects for which the research is carried out at a swiss institution
data_filtered = data[data['University'] != ]

In [ ]:
len(data_filtered)

In [ ]:
def split_date(x):
    x = x.split('.')
    return x[-1]

In [ ]:
data_filtered['Start Date'].apply(split_date)#[0].split('.')

In [ ]:
api.geonames.org/search?q=Facoltà_di_Teologia_di_Lugano&max_row=10&username=patk